In [14]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from alu import ArithmeticAttentionModel

In [16]:
model = ArithmeticAttentionModel()
# model = torch.compile(model)
torch.set_float32_matmul_precision('high')

In [17]:
num1 = torch.tensor([[1]]).float()
num2 = torch.tensor([[2]]).float()
op = torch.tensor([[0, 0, 1, 0]]).float()

In [18]:
model(num1, num2, op)

tensor([[13307.4150]], grad_fn=<SumBackward1>)

In [19]:
def arithmetic_loss(predictions, targets, scale_factor=10000.0):
    abs_error = (predictions - targets)**2
    # rel_error = torch.abs((predictions - targets) / (targets + 1e-8)) * scale_factor
    loss = abs_error # + rel_error
    return torch.sum(loss)

In [20]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset

class ArithmeticDataset(IterableDataset):
    def __init__(self, min_val=0, max_val=256):
        self.min_val = min_val
        self.max_val = max_val
        
        self.operations = {
            0: lambda x, y: x + y,    # addition
            1: lambda x, y: x - y,    # subtraction
            2: lambda x, y: x * y,    # multiplication
            3: lambda x, y: x / (y + 1e-8)  # division
        }
    
    def __iter__(self):
        while True:
            # Generate random numbers
            num1 = torch.rand(1) * (self.max_val - self.min_val) + self.min_val
            num2 = torch.rand(1) * (self.max_val - self.min_val) + self.min_val
            
            # Generate random operations
            op_idx = torch.tensor([0]) # torch.randint(0, 4, (1,))
            operation = F.one_hot(op_idx, num_classes=4).float()
            
            # Calculate targets
            target = self.operations[op_idx.item()](num1, num2)            
            
            yield num1, num2, operation.squeeze(0), target

In [21]:
ad = ArithmeticDataset()
print(next(iter(ad)))
dataloader = torch.utils.data.DataLoader(ad, batch_size=2)
print(next(iter(dataloader)))

(tensor([49.4587]), tensor([241.3123]), tensor([1., 0., 0., 0.]), tensor([290.7710]))
[tensor([[159.4502],
        [ 26.7862]]), tensor([[ 14.7039],
        [156.6797]]), tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.]]), tensor([[174.1542],
        [183.4658]])]


In [22]:
import sys
!{sys.executable} -m pip install wandb
!wandb login

wandb: Currently logged in as: avivekanand. Use `wandb login --relogin` to force relogin


In [23]:
from tqdm import tqdm
import numpy as np
import torch
import pandas as pd
import wandb
from datetime import datetime
from torch.utils.data import Dataset, IterableDataset
from torch.utils.data import DataLoader
wandb.require("service")

def train_model(
    model,
    num_epochs=6000,
    batch_size=1024,
    initial_lr=1e-3,
    device='cuda',
    # eval_every=500,
    use_wandb=False,
    project_name="arithmetic_training"
):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=initial_lr)
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.7)
    
    dataset = ArithmeticDataset()
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=24, pin_memory=True, persistent_workers=True)
    
    steps_per_epoch = 1000
    best_loss = float('inf')
    
    # Initialize logging
    if use_wandb:
        wandb.init(project=project_name)
        wandb.config.update({
            "learning_rate": initial_lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "scheduler_step_size": 200,
            "scheduler_gamma": 0.7
        })
    else:
        # Create CSV log file with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = f'training_log_{timestamp}.csv'
        log_data = []
    
    for epoch in range(num_epochs):
        model.train()
        epoch_losses = []
        epoch_diffs = []
        
        data_iter = iter(dataloader)
        pbar = tqdm(range(steps_per_epoch), desc=f'Epoch {epoch+1}/{num_epochs}')
        for step in pbar:
            try:
                batch = next(data_iter)
            except StopIteration:
                data_iter = iter(dataloader)
                batch = next(data_iter)
            
            num1, num2, operation, targets = [item.to(device) for item in batch]
            
            # num1 = num1.unsqueeze(1)
            # num2 = num2.unsqueeze(1)
            
            optimizer.zero_grad()
            predictions = model(num1, num2, operation)
            loss = arithmetic_loss(predictions, targets)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_losses.append(loss.item())
            
            with torch.no_grad():
                diffs = torch.abs(predictions - targets)
                epoch_diffs.extend(diffs.cpu().numpy())
            
            pbar.set_postfix({'Loss': loss.item()})
        
        with torch.no_grad():
            model.eval()
            test_num1, test_num2, test_op, test_targets = [item.to(device) for item in next(iter(dataloader))]
            
            test_pred = model(test_num1, test_num2, test_op)
            test_loss = arithmetic_loss(test_pred, test_targets)
           
            first_pred = test_pred[0].item()
            first_target = test_targets[0].item()
            
            # Format to 5 decimal places
            first_pred_formatted = f"{first_pred:.5f}"
            first_target_formatted = f"{first_target:.5f}"
            
            current_lr = optimizer.param_groups[0]['lr']
            train_loss = np.mean(epoch_losses)
            val_loss = test_loss.item()
            avg_diff = np.mean(epoch_diffs)
            median_diff = np.median(epoch_diffs)
            
            if use_wandb:
                wandb.log({
                    'learning_rate': current_lr,
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'avg_prediction_diff': avg_diff,
                    'median_prediction_diff': median_diff,
                    'epoch': epoch + 1
                })
            else:
                log_data.append({
                    'epoch': epoch + 1,
                    'learning_rate': current_lr,
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'avg_prediction_diff': avg_diff,
                    'median_prediction_diff': median_diff
                })
            
            print(
                f'Epoch {epoch+1}/{num_epochs} | '
                f'LR: {current_lr:.2e} | '
                f'Train Loss: {train_loss:.4f} | '
                f'Val Loss: {val_loss:.4f} | '
                f'Avg Diff: {avg_diff:.4f} | '
                f'First Pred: {first_pred_formatted} | '
                f'First Target: {first_target_formatted}'
            )
        
        model.train()
        
        # Save the best model
        if train_loss < best_loss:
            best_loss = train_loss
            torch.save(model.state_dict(), 'best_arithmetic_model.pt')
        
        scheduler.step()
        print(f'Epoch {epoch+1} completed. Average loss: {train_loss:.4f}\n')
    
    if not use_wandb:
        pd.DataFrame(log_data).to_csv(log_file, index=False)
        print(f"Training log saved to {log_file}")
    
    if use_wandb:
        wandb.finish()

In [ ]:
train_model(model, num_epochs=8000, batch_size=1024, initial_lr=1e-4, device='cuda', use_wandb=True)

Epoch 1010/8000: 100%|██████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 153.43it/s, Loss=400]


Epoch 1010/8000 | LR: 1.68e-05 | Train Loss: 166.2459 | Val Loss: 261.3722 | Avg Diff: 0.2923 | First Pred: 332.68600 | First Target: 333.09430
Epoch 1010 completed. Average loss: 166.2459



Epoch 1011/8000: 100%|██████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 153.39it/s, Loss=659]


Epoch 1011/8000 | LR: 1.68e-05 | Train Loss: 169.1221 | Val Loss: 240.1732 | Avg Diff: 0.2941 | First Pred: 386.65146 | First Target: 386.15509
Epoch 1011 completed. Average loss: 169.1221



Epoch 1012/8000: 100%|██████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 153.19it/s, Loss=396]


...

Epoch 1045/8000: 100%|██████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 151.41it/s, Loss=461]


Epoch 1045/8000 | LR: 1.68e-05 | Train Loss: 145.4521 | Val Loss: 269.4271 | Avg Diff: 0.2754 | First Pred: 217.35196 | First Target: 216.80904
Epoch 1045 completed. Average loss: 145.4521



Epoch 1046/8000: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 152.94it/s, Loss=14.4]


Epoch 1046/8000 | LR: 1.68e-05 | Train Loss: 184.8154 | Val Loss: 448.9686 | Avg Diff: 0.3097 | First Pred: 169.06206 | First Target: 169.27998
Epoch 1046 completed. Average loss: 184.8154



Epoch 1047/8000:  60%|█████████████████████████████████████▊                         | 600/1000 [00:04<00:02, 153.94it/s, Loss=266]